In [14]:
import base64
import cv2
import numpy as np
import redis
import pickle
import multiprocessing
import logging

In [15]:
class Reader:

    def __init__(self):
        self.__store = redis.Redis()
    
    def reader(self, image_key, shape_key):
        while True:
            encoded_image = self.__store.get(image_key)
            encoded_image_shape = pickle.loads(self.__store.get(shape_key))

            decoded_image = base64.b64decode(encoded_image)
            decoded_image = np.frombuffer(decoded_image, dtype=np.uint8)
            reshaped_decoded_image = decoded_image.reshape(encoded_image_shape)

            cv2.imshow("Video From Redis", reshaped_decoded_image)
            k = cv2.waitKey(1)

            if k == 27:
                cv2.destroyAllWindows()
                break
            
        cv2.destroyAllWindows()
    
    def run(self):
        image_key_list = ["cam_0", "cam_1"]
        image_shape_key_list = ["shape_0", "shape_1"]

        jobs = []
        for image_key, shape_key in zip(image_key_list, image_shape_key_list):
            p = multiprocessing.Process(target=self.reader, args=(image_key, shape_key,))
            p.start()
            jobs.append(p)

        
        for p in jobs:
            p.join()

In [16]:
if __name__ == '__main__':
	obj = Reader()
	obj.run()